In [ ]:
# ================================================================
# ADVANCED TIME SERIES FORECASTING WITH HTM-LIKE MODEL + LSTM
# FULL IMPLEMENTATION + DELIVERABLE #2 (WRITTEN ANALYSIS)
# Saves: deliverable_2_analysis.md, comparison_plot.png
# ================================================================

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import datetime

# ----------------------------
# Configuration / Hyperparams
# ----------------------------
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Paths (screenshot provided by user; include as-is)
USER_SCREENSHOT_PATH = "/mnt/data/a7d056bf-87b0-4a7a-b544-ac7462c08d44.png"
ANALYSIS_OUTPUT = "deliverable_2_analysis.md"
PLOT_OUTPUT = "comparison_plot.png"

# ----------------------------
# 1) GENERATE MULTIVARIATE TIME SERIES DATA
# ----------------------------
def generate_data(n=12000):
    """Generate multivariate seasonal + trend time-series."""
    t = np.arange(n).astype(float)

    # Multiple components to create complex series:
    # - x1: seasonality + slow upward trend + noise
    # - x2: a lower amplitude seasonal component + noise
    # - x3: interaction / higher frequency term
    x1 = 0.5 * np.sin(0.02 * t) + 0.001 * t + np.random.normal(0, 0.1, n)
    x2 = 0.3 * np.cos(0.015 * t) + np.random.normal(0, 0.05, n)
    x3 = 0.2 * np.sin(0.05 * t) * np.cos(0.01 * t) + np.random.normal(0, 0.03, n)

    df = pd.DataFrame({"x1": x1, "x2": x2, "x3": x3})
    return df

data = generate_data(n=12000)
print(f"[INFO] Generated data shape: {data.shape}")

# Scale data for LSTM and for SDR encoding mapping convenience
scaler = MinMaxScaler()
scaled = scaler.fit_transform(data)
scaled_df = pd.DataFrame(scaled, columns=data.columns)

# ----------------------------
# 2) SIMPLE SDR ENCODER (HTM Input)
# ----------------------------
def sdr_encode(value, n_bits=256, active_bits=20):
    """Convert scalar value in [0,1] -> Sparse Distributed Representation (binary vector)."""
    # clamp value
    v = float(np.clip(value, 0.0, 1.0))
    sdr = np.zeros(n_bits, dtype=np.int8)
    index = int(v * (n_bits - active_bits))
    index = max(0, min(index, n_bits - active_bits))
    sdr[index:index + active_bits] = 1
    return sdr

def encode_dataset(df, n_bits_per_feature=256, active_bits=20):
    encoded = []
    for _, row in df.iterrows():
        enc = np.hstack([
            sdr_encode(row["x1"], n_bits=n_bits_per_feature, active_bits=active_bits),
            sdr_encode(row["x2"], n_bits=n_bits_per_feature, active_bits=active_bits),
            sdr_encode(row["x3"], n_bits=n_bits_per_feature, active_bits=active_bits)
        ])
        encoded.append(enc)
    return np.array(encoded)

# Use default SDR params (these are also recorded as critical hyperparams below)
SDR_BITS = 256
SDR_ACTIVE = 20
sdr_data = encode_dataset(scaled_df, n_bits_per_feature=SDR_BITS, active_bits=SDR_ACTIVE)
print(f"[INFO] SDR encoded data shape: {sdr_data.shape} (bits per sample)")

# ----------------------------
# 3) HTM-LIKE TEMPORAL MEMORY MODEL
# ----------------------------
class HTMModel:
    """A simplified HTM-like temporal memory using a memory matrix and Hebbian-like updates."""

    def __init__(self, input_size, mem_cells=800, sparsity=0.03, lr=0.002):
        self.input_size = input_size
        self.mem_cells = mem_cells
        self.lr = lr
        # Memory matrix (mem_cells x input_size)
        self.memory = np.random.rand(mem_cells, input_size).astype(np.float32)
        # How many cells to activate (sparse representation of memory)
        self.k = max(1, int(sparsity * mem_cells))
        self.sparsity = sparsity

    def predict(self, x):
        """
        Compute similarity between input x and memory rows.
        Return a binary sparse activation vector of size mem_cells with k active cells.
        """
        # raw similarity (dot product)
        sim = self.memory @ x
        # pick top-k
        top_indices = np.argpartition(sim, -self.k)[-self.k:]
        activation = np.zeros(self.mem_cells, dtype=np.float32)
        activation[top_indices] = 1.0
        return activation

    def learn(self, x, y_pred):
        """Simple Hebbian-like update: strengthen memory rows (outer product) and clip."""
        self.memory += self.lr * np.outer(y_pred, x)
        # keep memory in reasonable range
        np.clip(self.memory, 0.0, 1.0, out=self.memory)

    def run_sequence(self, data):
        """Run through sequence and learn online; return activations (predictions) per step."""
        preds = []
        for i in range(len(data) - 1):
            pred = self.predict(data[i])
            self.learn(data[i], pred)
            preds.append(pred)
        return np.array(preds)

# instantiate HTM
htm_params = {
    "mem_cells": 800,
    "sparsity": 0.03,
    "learning_rate": 0.002,
    "sdr_bits_per_feature": SDR_BITS,
    "active_bits": SDR_ACTIVE
}
htm = HTMModel(input_size=sdr_data.shape[1],
               mem_cells=htm_params["mem_cells"],
               sparsity=htm_params["sparsity"],
               lr=htm_params["learning_rate"])

# run HTM to generate SDR-form predictions
htm_preds_sdr = htm.run_sequence(sdr_data)
print(f"[INFO] HTM produced SDR predictions shape: {htm_preds_sdr.shape}")

# decode SDR-like memory activation -> scalar proxy
def decode_sdr_to_scalar(sdr_vec):
    """Map an activation vector to a scalar between 0 and 1 by using the index of maximum activation."""
    # We map the index of the maximum active cell to the [0,1] range
    idx = int(np.argmax(sdr_vec))
    return idx / max(1, len(sdr_vec) - 1)

htm_output_raw = np.array([decode_sdr_to_scalar(p) for p in htm_preds_sdr])

# Map HTM scalar proxy back to original x1 scale for comparison
def map_proxy_to_series(proxy, reference_series):
    # normalize proxy to [0,1]
    pmin, pmax = proxy.min(), proxy.max()
    if pmax > pmin:
        pn = (proxy - pmin) / (pmax - pmin)
    else:
        pn = np.zeros_like(proxy)
    # map to reference_series min/max
    ref_min, ref_max = reference_series.min(), reference_series.max()
    mapped = pn * (ref_max - ref_min) + ref_min
    return mapped

# ----------------------------
# 4) LSTM BASELINE MODEL
# ----------------------------
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

def create_sequences(npdata, seq_len=20):
    X, y = [], []
    for i in range(len(npdata) - seq_len):
        X.append(npdata[i:i+seq_len])
        y.append(npdata[i+seq_len])
    return np.array(X), np.array(y)

SEQ_LEN = 20
X, y = create_sequences(scaled, seq_len=SEQ_LEN)
print(f"[INFO] LSTM sequence dataset shapes: X={X.shape}, y={y.shape}")

# convert to torch tensors
X_t = torch.tensor(X, dtype=torch.float32)
y_t = torch.tensor(y, dtype=torch.float32)

train_ds = TensorDataset(X_t, y_t)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)

# LSTM model hyperparams
lstm_hparams = {
    "input_dim": 3,
    "hidden_dim": 64,
    "layers": 2,
    "lr": 0.001,
    "epochs": 12
}

device = torch.device("cpu")
model = LSTMModel(input_dim=lstm_hparams["input_dim"],
                  hidden_dim=lstm_hparams["hidden_dim"],
                  layers=lstm_hparams["layers"]).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lstm_hparams["lr"])
loss_fn = nn.MSELoss()

# Train LSTM
print("[INFO] Training LSTM...")
for epoch in range(lstm_hparams["epochs"]):
    model.train()
    running_loss = 0.0
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        optimizer.zero_grad()
        pred = model(batch_x)
        loss = loss_fn(pred, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * batch_x.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"  Epoch {epoch+1}/{lstm_hparams['epochs']}: loss = {epoch_loss:.6f}")

# LSTM predictions on whole input set
model.eval()
with torch.no_grad():
    lstm_preds_scaled = model(X_t.to(device)).cpu().numpy()

# invert scaling for LSTM outputs to original value scale
lstm_preds = scaler.inverse_transform(lstm_preds_scaled)

# ----------------------------
# 5) EVALUATION
# ----------------------------
# Compare predictions for x1 only (commonly one target, while model predicts all features)
true_values = data["x1"].values[SEQ_LEN: SEQ_LEN + len(lstm_preds)]
lstm_eval = lstm_preds[:, 0]

# HTM predictions mapped to x1 scale. Use the same length as true_values
htm_eval_raw = htm_output_raw[:len(true_values)]
htm_eval = map_proxy_to_series(htm_eval_raw, data["x1"].values)

def evaluate(true, pred):
    return {
        "RMSE": float(np.sqrt(mean_squared_error(true, pred))),
        "MAE": float(mean_absolute_error(true, pred))
    }

metrics_htm = evaluate(true_values, htm_eval)
metrics_lstm = evaluate(true_values, lstm_eval)

print("\n========== MODEL COMPARISON ==========")
print("HTM Model Performance:", metrics_htm)
print("LSTM Baseline Performance:", metrics_lstm)

# directional accuracy (optional quick metric)
direction_true = np.sign(true_values[1:] - true_values[:-1])
direction_htm = np.sign(htm_eval[1:] - htm_eval[:-1])
direction_lstm = np.sign(lstm_eval[1:] - lstm_eval[:-1])

dir_acc_htm = float((direction_true == direction_htm).mean())
dir_acc_lstm = float((direction_true == direction_lstm).mean())
print(f"Direction accuracy — HTM: {dir_acc_htm:.4f}, LSTM: {dir_acc_lstm:.4f}")

# ----------------------------
# 6) SAVE COMPARISON PLOT
# ----------------------------
plt.figure(figsize=(12,6))
plt.plot(true_values, label="True x1 (target)", linewidth=1)
plt.plot(lstm_eval, label="LSTM prediction (x1)", linewidth=1)
plt.plot(htm_eval, label="HTM prediction (mapped proxy)", linewidth=1)
plt.legend()
plt.title("HTM vs LSTM Forecast Comparison")
plt.xlabel("Time step (test slice)")
plt.ylabel("x1 value")
plt.tight_layout()
plt.savefig(PLOT_OUTPUT, dpi=150)
plt.close()
print(f"[INFO] Saved comparison plot to: {PLOT_OUTPUT}")

# ----------------------------
# 7) DELIVERABLE #2: WRITTEN ANALYSIS (saved as markdown)
# ----------------------------
now = datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")
analysis_lines = []

analysis_lines.append(f"# Deliverable #2 — Written Analysis\n")
analysis_lines.append(f"**Generated:** {now}\n")
analysis_lines.append(f"**Dataset:** synthetic multivariate time series (12,000 observations, 3 features)\n")
analysis_lines.append(f"**Screenshot reference (provided by user):** `{USER_SCREENSHOT_PATH}`\n")
analysis_lines.append("\n---\n")

# HTM architecture choices
analysis_lines.append("## 1. HTM Architecture Choices and Rationale\n")
analysis_lines.append("- **SDR Encoding**: Each scalar feature is encoded into a Sparse Distributed Representation (SDR) with "
                      f"{SDR_BITS} bits and {SDR_ACTIVE} active bits. SDRs are robust to noise and support similarity-based retrieval.\n")
analysis_lines.append("- **Memory Matrix**: The HTM-like model uses a memory matrix with `mem_cells` rows (here: "
                      f"{htm_params['mem_cells']}) representing learned prototypical contexts. Each row learns associations to input SDRs.\n")
analysis_lines.append("- **Sparse Activation (k active cells)**: A small fraction (sparsity={htm_params['sparsity']}) of memory cells are activated per input. "
                      "This enforces distributed, sparse representations similar to canonical HTM designs.\n")
analysis_lines.append("- **Learning Rule**: A Hebbian-like outer-product update strengthens memory rows which were activated by an input (`learning_rate`="
                      f"{htm_params['learning_rate']}). This simple rule is a lightweight proxy to temporal memory adaptation.\n")
analysis_lines.append("- **Prediction Readout**: Because this is a simplified HTM-style implementation, predictions are derived from memory activations (index of max activation) "
                      "and mapped back to the scalar domain for comparison with the LSTM baseline.\n")
analysis_lines.append("\n")

# Hyperparameter tuning strategy
analysis_lines.append("## 2. Hyperparameter Tuning Strategy\n")
analysis_lines.append("The following hyperparameters were considered and tuned informally (grid / manual search):\n")
analysis_lines.append(f"- `mem_cells` (memory capacity): {htm_params['mem_cells']} — larger memory can store more contexts, but increases compute.\n")
analysis_lines.append(f"- `sparsity` (fraction of active memory cells): {htm_params['sparsity']} — controls representation sparsity and overlap.\n")
analysis_lines.append(f"- `learning_rate` (Hebbian update multiplier): {htm_params['learning_rate']} — affects plasticity; small values prevent catastrophic overwrite.\n")
analysis_lines.append(f"- `sdr_bits_per_feature`: {htm_params['sdr_bits_per_feature']} — higher resolution SDRs increase discriminability at cost of dimension.\n")
analysis_lines.append(f"- `active_bits`: {htm_params['active_bits']} — determines how many bits are on in each SDR; affects robustness to noise.\n")
analysis_lines.append("\n")
analysis_lines.append("**Tuning approach used in this deliverable:**\n")
analysis_lines.append("- Manual, pragmatic tuning: we selected a balanced memory size (800 cells) and low sparsity (3%) to capture recurring contexts while keeping the representation sparse.\n")
analysis_lines.append("- For a production experiment, automated tuning (e.g., randomized search or Bayesian optimization) across `mem_cells`, `sparsity`, and `learning_rate` would be recommended with cross-validation on held-out segments.\n")
analysis_lines.append("\n")

# Side-by-side comparison
analysis_lines.append("## 3. Side-by-side Performance Comparison\n")
analysis_lines.append("| Model | RMSE (x1) | MAE (x1) | Direction Accuracy |\n")
analysis_lines.append("|---|---:|---:|---:|\n")
analysis_lines.append(f"| HTM-like model | {metrics_htm['RMSE']:.6f} | {metrics_htm['MAE']:.6f} | {dir_acc_htm:.4f} |\n")
analysis_lines.append(f"| LSTM baseline   | {metrics_lstm['RMSE']:.6f} | {metrics_lstm['MAE']:.6f} | {dir_acc_lstm:.4f} |\n")
analysis_lines.append("\n")
analysis_lines.append("**Observations:**\n")
analysis_lines.append("- The LSTM (a parametric neural sequence model) typically provides smoother, directly regressed numeric outputs and often achieves lower RMSE/MAE on continuous-scalar forecasting tasks when ample data and training are available.\n")
analysis_lines.append("- The HTM-like model here is a simplified, biologically-inspired online memory mechanism. It excels at encoding recurring contexts and retrieving them but requires careful mapping from sparse activations back to scalar predictions; this mapping is inherently lossy compared to direct numeric regression.\n")
analysis_lines.append("\n")

# Interpretation + limitations
analysis_lines.append("## 4. Interpretation of Results\n")
analysis_lines.append("- If LSTM RMSE/MAE is lower than HTM, it indicates that the parameterized sequence model better captures the numeric mapping for this synthetic dataset under the current training regime.\n")
analysis_lines.append("- HTM-style systems are strong in online, continual-learning scenarios with abrupt context changes or where explainability of pattern recall is required.\n")
analysis_lines.append("\n")
analysis_lines.append("## 5. Limitations of This Implementation\n")
analysis_lines.append("- This HTM-like implementation is a simplified approximation (no column/cell structure, no distal segment permanence, no temporal pooling). It's intended for educational and comparative purposes, not as a full NuPIC replacement.\n")
analysis_lines.append("- The decode-from-activation-to-scalar approach (argmax index mapping) is a coarse readout; more sophisticated decoders (learned linear or non-linear readouts) would likely improve HTM numeric predictions.\n")
analysis_lines.append("- Hyperparameter tuning here is manual. Robust evaluation requires systematic cross-validation and a proper validation set, especially for real-world datasets.\n")
analysis_lines.append("\n")

# 6. Recommendations / Next steps
analysis_lines.append("## 6. Recommendations & Next Steps\n")
analysis_lines.append("- Replace the HTM argmax readout with a learned regression head that maps sparse activation patterns to continuous targets (e.g., fit a ridge/regression or small neural net on top of memory activations).\n")
analysis_lines.append("- Use automated hyperparameter optimization (Optuna / Ray Tune) with time series cross-validation to find optimal `mem_cells`, `sparsity`, and `learning_rate`.\n")
analysis_lines.append("- Experiment with different SDR encoders (e.g., overlapping encoders, multi-scale encoders) and with temporal pooling to capture longer contexts.\n")
analysis_lines.append("- Test on real-world benchmarks (e.g., M4, electricity, traffic) and compare HTM-like models vs LSTM/Transformer baselines.\n")
analysis_lines.append("\n")

# 7. Top 5 critical HTM hyperparameters (explicit)
analysis_lines.append("## 7. Top 5 Critical HTM Hyperparameters\n")
for k, v in htm_params.items():
    analysis_lines.append(f"- **{k}**: {v}\n")
analysis_lines.append("\n---\n")
analysis_lines.append(f"## Files produced by this script\n")
analysis_lines.append(f"- `{ANALYSIS_OUTPUT}` — this written analysis in markdown\n")
analysis_lines.append(f"- `{PLOT_OUTPUT}` — comparison plot (True vs LSTM vs HTM)\n")
analysis_lines.append(f"- Screenshot reference provided by user: `{USER_SCREENSHOT_PATH}` (convert to URL as needed)\n")

# Write markdown file
with open(ANALYSIS_OUTPUT, "w", encoding="utf-8") as f:
    f.write("\n".join(analysis_lines))

print(f"[INFO] Written analysis deliverable to: {ANALYSIS_OUTPUT}")

# Also print a short console summary for quick inspection
print("\n--- Quick Summary ---")
print(f"HTM RMSE: {metrics_htm['RMSE']:.6f}, MAE: {metrics_htm['MAE']:.6f}")
print(f"LSTM RMSE: {metrics_lstm['RMSE']:.6f}, MAE: {metrics_lstm['MAE']:.6f}")
print(f"Deliverable file: {ANALYSIS_OUTPUT}")
print(f"Plot file: {PLOT_OUTPUT}")
print(f"Screenshot (user-provided): {USER_SCREENSHOT_PATH}")

# End of script
